In [1]:
import os
import sys
import json
import codecs

import time
from datetime import datetime

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark import SparkFiles
from pyspark.sql.functions import col, lit, length, row_number, when

In [3]:
from lib import spark_utils

In [4]:
spark = spark_utils.get_spark()

25/03/08 11:44:58 WARN Utils: Your hostname, Mac-MD2XX1D4WV.local resolves to a loopback address: 127.0.0.1; using 192.168.10.30 instead (on interface en0)
25/03/08 11:44:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 11:44:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/08 11:44:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark

In [6]:
data = [(1, "Alice"), (2, "Bob")]
columns = ["id", "name"]
df = spark.createDataFrame(data, columns)

In [7]:
cur_year = 1
cur_qurt = 2
pre_year = 3
pre_qurt = 4

In [15]:
df =  df.withColumns({
    'YEAR': lit(cur_year),
    'QUARTER': lit(cur_qurt),
    'DATA_YEAR': lit(pre_year),
    'DATA_QURT': lit(pre_qurt),
})

In [16]:
df.show()

+---+-----+----+-------+---------+---------+
| id| name|YEAR|QUARTER|DATA_YEAR|DATA_QURT|
+---+-----+----+-------+---------+---------+
|  1|Alice|   1|      2|        3|        4|
|  2|  Bob|   1|      2|        3|        4|
+---+-----+----+-------+---------+---------+



In [12]:
input_file = '/Users/liuda/Local/data/trading/data/hedgeparsing/2021q4_form13f/ticker/part*'

In [27]:
schema = StructType([
    StructField("CUSIP", StringType(), nullable=True),
    StructField("MANAGER_LEVEL", StringType(), nullable=True),
    StructField("INC_VALUE", LongType(), nullable=True),
    StructField("DRC_VALUE", LongType(), nullable=True),
    StructField("TOTAL_VALUE", LongType(), nullable=True),
    StructField("TOTAL_PREVALUE", LongType(), nullable=True),
    StructField("SYMBOL", StringType(), nullable=True),
])

ticker_data = spark.read.schema(schema).option('delimiter', '\t').option('header', True).csv(input_file)

In [28]:
ticker_data.filter(col('SYMBOL').isNotNull()).show(3, False)

+---------+-------------+---------+---------+-----------+--------------+------+
|CUSIP    |MANAGER_LEVEL|INC_VALUE|DRC_VALUE|TOTAL_VALUE|TOTAL_PREVALUE|SYMBOL|
+---------+-------------+---------+---------+-----------+--------------+------+
|009126202|1            |44000    |-3917000 |2333000    |6206000       |AIQUY |
|013091103|4            |23934000 |-10529000|63200000   |49795000      |ACI   |
|025072877|2            |260000   |0        |512000     |252000        |AVUV  |
+---------+-------------+---------+---------+-----------+--------------+------+
only showing top 3 rows



In [29]:
ticker_data = ticker_data.withColumns({
    'DIFF_VALUE': col('INC_VALUE') + col('DRC_VALUE')
})

In [30]:
ticker_data.show(3, False)

+---------+-------------+---------+----------+-----------+--------------+------+----------+
|CUSIP    |MANAGER_LEVEL|INC_VALUE|DRC_VALUE |TOTAL_VALUE|TOTAL_PREVALUE|SYMBOL|DIFF_VALUE|
+---------+-------------+---------+----------+-----------+--------------+------+----------+
|00208d408|2            |20000    |0         |221000     |201000        |NULL  |20000     |
|006985383|2            |0        |-137524000|908246000  |1045770000    |NULL  |-137524000|
|007025109|2            |1101000  |-22000    |1144000    |65000         |NULL  |1079000   |
+---------+-------------+---------+----------+-----------+--------------+------+----------+
only showing top 3 rows



In [32]:
ticker_data = ticker_data.withColumns({
    # 当前qt的情况
    'INC_VALUE': when(col('DIFF_VALUE').isNotNull() & (col('DIFF_VALUE') > 0), col('DIFF_VALUE')).otherwise(0),
    'DRC_VALUE': when(col('DIFF_VALUE').isNotNull() & (col('DIFF_VALUE') < 0), col('DIFF_VALUE')).otherwise(0),
    'INC_COUNT': sum(when(col('DIFF_VALUE') > 0, 1).otherwise(0)),
    'DRC_COUNT': sum(when(col('DIFF_VALUE') < 0, 1).otherwise(0)),
})